In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

In [3]:
#Question 1
user_question = "I just discovered the course. Can I still join it?"
text_emb = embedding_model.encode(user_question)
text_emb[0]

0.07822268

In [4]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

documents = list(filter(lambda doc: doc['course'] == 'machine-learning-zoomcamp', documents))
print(len(documents))

375


In [5]:
embeddings = []
for document in tqdm(documents):
    question = document['question']
    text = document['text']
    qa_text = f'{question} {text}'
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

  0%|          | 0/375 [00:00<?, ?it/s]

In [6]:
# Question 2
X = np.array(embeddings)
print(X.shape)

(375, 768)


In [7]:
# Question 3
scores = X.dot(text_emb)
print(np.max(scores, axis=0))

0.6506573


In [8]:
#Question 4

class VectorSearchEngine:
    def __init__(self, _documents, _embeddings):
        self.documents = _documents
        self.embeddings = _embeddings

    def search(self, v_query, num_results=10):
        _scores = self.embeddings.dot(v_query)
        idx = np.argsort(-_scores)[:num_results]
        return [self.documents[i] for i in idx]
    
search_engine = VectorSearchEngine(_documents=documents, _embeddings=X)

In [9]:
search_results = search_engine.search(text_emb, num_results=5)

In [10]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [11]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [12]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank]:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [13]:
def evaluate(_ground_truth, search_function):
    relevance_total = []

    for q in tqdm(_ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [14]:
def custom_vector_search(q):
    v_q = embedding_model.encode(q['question'])

    return search_engine.search(v_q, num_results=5)

In [15]:
print(evaluate(ground_truth, custom_vector_search))

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}


In [16]:
#Question 5

from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 
es_client.ping()

True

In [17]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

/var/folders/py/hx_rr7lj0yj98b1zv61tqlkm0000gn/T/ipykernel_61514/505606794.py:26: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [18]:
for idx, doc in enumerate(documents):
    doc['vector'] = embeddings[idx]

In [19]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [20]:


def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000
    }

    es_results = es_client.search(
        index=index_name,
        knn=knn,
        _source=["text", "section", "question", "course", "id"]
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        hit['_source']['score'] = hit['_score']
        result_docs.append(hit['_source'])

    return result_docs

In [21]:
elastic_search_knn('vector', text_emb)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693',
  'score': 0.82532895},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first laun

In [22]:
#Question 6

def es_vector_search(q):
    v_q = embedding_model.encode(q['question'])
    return elastic_search_knn('vector', v_q)

In [23]:
print(evaluate(ground_truth, es_vector_search))

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}
